In [1]:
# %load model.py
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import time
import numpy as np
import tensorflow as tf

VGG_MEAN = [103.939, 116.779, 123.68] # [B, G, R]
class VGG16:
    def __init__(self, scope_name="VGG16"):
        """
        load pre-trained weights from path
        :param vgg16_npy_path: file path of vgg16 pre-trained weights
        """
        
        self.scope_name = scope_name
        
        self.gamma_var = []
        self.net_shape = []
        
        # operation dictionary
        self.prob_dict = {}
        self.loss_dict = {}
        self.accu_dict = {}
        self.feature_dict = {}

        # parameter dictionary
        self.para_dict = {}

    def build(self, vgg16_npy_path, classes=10, shape=(32,32,3), prof_type=None, conv_pre_training=True, fc_pre_training=True):
        """
        load variable from npy to build the VGG
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """
        
        # input information
        self.H, self.W, self.C = shape
        self.classes = classes
        
        start_time = time.time()
        print("build model started")

        if prof_type is None:
            self.prof_type = "all-one"
        else:
            self.prof_type = prof_type

        # load pre-trained weights
        if isinstance(vgg16_npy_path,dict):
            self.data_dict = vgg16_npy_path
            print("parameters loaded")
        else:
            self.data_dict = np.load(vgg16_npy_path, encoding='latin1').item()
            print("npy file loaded")

        # input placeholder
        self.x = tf.placeholder(tf.float32, [None, self.H, self.W, self.C])
        self.y = tf.placeholder(tf.float32, [None, self.classes])
        self.is_train = tf.placeholder(tf.bool)
        
        self.x = tf.placeholder(tf.float32, [None, self.H, self.W, self.C])
        self.is_train = tf.placeholder(tf.bool)
        
        # Convert RGB to BGR
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=self.x)

        self.x = tf.concat(axis=3, values=[
             blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
              red - VGG_MEAN[2],
        ])
        assert self.x.get_shape().as_list()[1:] == [self.H, self.W, self.C]

        dp={
            'conv1_1':1.00,
            'conv1_2':1.00,
            'conv2_1':1.00,
            'conv2_2':1.00,
            'conv3_1':1.00,
            'conv3_2':1.00,
            'conv3_3':1.00,
            'conv4_1':1.00,
            'conv4_2':1.00,
            'conv4_3':1.00,
            'conv5_1':1.00,
            'conv5_2':1.00,
            'conv5_3':1.00
            }

        # declare and initialize the weights of VGG16
        with tf.variable_scope(self.scope_name):
            # weight decay
            self._weight_decay = 0.0
            for k, v in sorted(dp.items()):
                (conv_filter, gamma, beta, bn_mean, bn_variance), conv_bias = self.get_conv_filter(name=k, new_bn=True), self.get_bias(name=k)
                self.para_dict[k] = [conv_filter, conv_bias]
                self.para_dict[k+"_gamma"] = gamma
                self.para_dict[k+"_beta"] = beta
                self.para_dict[k+"_bn_mean"] = bn_mean
                self.para_dict[k+"_bn_variance"] = bn_variance
                self.gamma_var.append(self.para_dict[k+"_gamma"])

                # weight decay
                self._weight_decay += tf.nn.l2_loss(conv_filter)+tf.nn.l2_loss(conv_bias)

            if fc_pre_training:
                fc_W, fc_b = self.get_fc_layer(name='fc_2'), self.get_bias(name='fc_2')
                self.para_dict['fc_2'] = [fc_W, fc_b]

                self._weight_decay += tf.nn.l2_loss(fc_W) + tf.nn.l2_loss(fc_b)
            else:
                # the last fully connected layers should be trained
                # user specified fully connected layers
                fc_W = self.get_fc_layer(name='fc_2', shape=(21504, self.classes))
                fc_b = self.get_bias(name='fc_2', shape=(self.classes,))
                self.para_dict['fc_2'] = [fc_W, fc_b]

                self._weight_decay += tf.nn.l2_loss(fc_W) + tf.nn.l2_loss(fc_b)

        print(("build model finished: %ds" % (time.time() - start_time)))

    def sparsity_train(self, l1_gamma=0.001, l1_gamma_diff=0.001, decay=0.0005, keep_prob=0.0):
        
        self._keep_prob = keep_prob
        start_time = time.time()
        with tf.name_scope("var_dp"):
            conv1_1 = self.idp_conv_bn_layer( self.x, "conv1_1")
            conv1_2 = self.idp_conv_bn_layer(conv1_1, "conv1_2")
            pool1 = self.max_pool(conv1_2, 'pool1')

            conv2_1 = self.idp_conv_bn_layer(  pool1, "conv2_1")
            conv2_2 = self.idp_conv_bn_layer(conv2_1, "conv2_2")
            pool2 = self.max_pool(conv2_2, 'pool2')

            conv3_1 = self.idp_conv_bn_layer(  pool2, "conv3_1")
            conv3_2 = self.idp_conv_bn_layer(conv3_1, "conv3_2")
            conv3_3 = self.idp_conv_bn_layer(conv3_2, "conv3_3")
            pool3 = self.max_pool(conv3_3, 'pool3')

            conv4_1 = self.idp_conv_bn_layer(  pool3, "conv4_1")
            conv4_2 = self.idp_conv_bn_layer(conv4_1, "conv4_2")
            conv4_3 = self.idp_conv_bn_layer(conv4_2, "conv4_3")
            pool4   = self.max_pool(conv4_3, 'pool4')

            conv5_1 = self.idp_conv_bn_layer(  pool4, "conv5_1")
            conv5_2 = self.idp_conv_bn_layer(conv5_1, "conv5_2")
            conv5_3 = self.idp_conv_bn_layer(conv5_2, "conv5_3")
            pool5 = self.max_pool(conv5_3, 'pool5')

            pool5 = self.dropout_layer(pool5, keep_prob=self._keep_prob)

            logits = self.fc_layer(pool5, 'fc_2')       
            prob = tf.nn.softmax(logits, name="prob")
            
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.y)
            loss = tf.reduce_mean(cross_entropy)
            accuracy = tf.reduce_mean(tf.cast(tf.equal(x=tf.argmax(logits, 1), y=tf.argmax(self.y, 1)),tf.float32))
            
            # gamma l1 regularization
            l1_gamma_regularizer = tf.contrib.layers.l1_regularizer(scale=l1_gamma)
            gamma_l1 = tf.contrib.layers.apply_regularization(l1_gamma_regularizer, self.gamma_var)

            # gamma_diff l1 regularization
            def non_increasing_constraint_axis_0(a):
                return tf.nn.relu(a[1:]-a[:-1])
            gamma_diff_var = [non_increasing_constraint_axis_0(x) for x in self.gamma_var]

            l1_gamma_diff_regularizer = tf.contrib.layers.l1_regularizer(scale=l1_gamma_diff)
            gamma_diff_l1 = tf.contrib.layers.apply_regularization(l1_gamma_diff_regularizer, gamma_diff_var)
            
            self.prob_dict["var_dp"] = prob
            self.loss_dict["var_dp"] = loss + gamma_l1 + gamma_diff_l1 + self._weight_decay * decay
            self.accu_dict["var_dp"] = accuracy
            
            tf.summary.scalar(name="accu_var_dp", tensor=accuracy)
            tf.summary.scalar(name="loss_var_dp", tensor=loss)
        self.summary_op = tf.summary.merge_all()        
        print(("sparsity train operation setup: %ds" % (time.time() - start_time)))
    
    def set_idp_operation(self, dp, decay=0.0002, keep_prob=1.0):
        self._keep_prob = keep_prob
        if type(dp) != list:
            raise ValueError("when block_variational is False, dp must be a list.")
        self.dp = dp 
        print("DP under test:", np.round(self.dp,2))
        start_time = time.time()
        # create operations at every dot product percentages
        for dp_i in dp:
            with tf.name_scope(str(int(dp_i*100))):
                conv1_1 = self.idp_conv_bn_layer( self.x, "conv1_1", dp_i)
                conv1_2 = self.idp_conv_bn_layer(conv1_1, "conv1_2", dp_i)
                pool1 = self.max_pool(conv1_2, 'pool1')

                if dp_i == 1.0:
                    self.net_shape.append(conv1_1.get_shape())
                    self.net_shape.append(pool1.get_shape())

                conv2_1 = self.idp_conv_bn_layer(  pool1, "conv2_1", dp_i)
                conv2_2 = self.idp_conv_bn_layer(conv2_1, "conv2_2", dp_i)
                pool2 = self.max_pool(conv2_2, 'pool2')

                if dp_i == 1.0:
                    self.net_shape.append(conv2_1.get_shape())
                    self.net_shape.append(pool2.get_shape())

                conv3_1 = self.idp_conv_bn_layer(  pool2, "conv3_1", dp_i)
                conv3_2 = self.idp_conv_bn_layer(conv3_1, "conv3_2", dp_i)
                conv3_3 = self.idp_conv_bn_layer(conv3_2, "conv3_3", dp_i)
                pool3 = self.max_pool(conv3_3, 'pool3')

                if dp_i == 1.0:
                    self.net_shape.append(conv3_1.get_shape())
                    self.net_shape.append(conv3_2.get_shape())
                    self.net_shape.append(pool3.get_shape())

                conv4_1 = self.idp_conv_bn_layer(  pool3, "conv4_1", dp_i)
                conv4_2 = self.idp_conv_bn_layer(conv4_1, "conv4_2", dp_i)
                conv4_3 = self.idp_conv_bn_layer(conv4_2, "conv4_3", dp_i)
                pool4 = self.max_pool(conv4_3, 'pool4')
                
                if dp_i == 1.0:
                    self.net_shape.append(conv4_1.get_shape())
                    self.net_shape.append(conv4_2.get_shape())
                    self.net_shape.append(pool4.get_shape())

                conv5_1 = self.idp_conv_bn_layer(  pool4, "conv5_1", dp_i)
                conv5_2 = self.idp_conv_bn_layer(conv5_1, "conv5_2", dp_i)
                conv5_3 = self.idp_conv_bn_layer(conv5_2, "conv5_3", dp_i)
                pool5 = self.max_pool(conv5_3, 'pool5')

                if dp_i == 1.0:
                    self.net_shape.append(conv5_1.get_shape())
                    self.net_shape.append(conv5_2.get_shape())
                    self.net_shape.append(pool5.get_shape())

#                 fc_1 = self.fc_layer(pool5, 'fc_1')
#                 fc_1 = self.dropout_layer(fc_1, keep_prob=self._keep_prob)
#                 fc_1 = tf.nn.relu(fc_1)
                
                logits = self.fc_layer(pool5, 'fc_2')
                prob = tf.nn.softmax(logits, name="prob")

                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.y)
                loss = tf.reduce_mean(cross_entropy)
                accuracy = tf.reduce_mean(tf.cast(tf.equal(x=tf.argmax(logits, 1), y=tf.argmax(self.y, 1)), dtype=tf.float32))

                # self.feature_dict[str(int(dp_i*100))] = fc_1
                self.prob_dict[str(int(dp_i*100))] = prob
                self.loss_dict[str(int(dp_i*100))] = loss + self._weight_decay * decay
                self.accu_dict[str(int(dp_i*100))] = accuracy

                tf.summary.scalar(name="accu_at_"+str(int(dp_i*100)), tensor=accuracy)
                tf.summary.scalar(name="loss_at_"+str(int(dp_i*100)), tensor=loss)
        self.summary_op = tf.summary.merge_all()
        print(("Set dp operations finished: %ds" % (time.time() - start_time)))

    def spareness(self, thresh=0.05):
        N_active, N_total = 0,0
        for gamma in self.gamma_var:
            m = tf.cast(tf.less(tf.abs(gamma), thresh), tf.float32)
            n_active = tf.reduce_sum(m)
            n_total  = tf.cast(tf.reduce_prod(tf.shape(m)), tf.float32)
            N_active += n_active
            N_total  += n_total
        return N_active/N_total

    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)
    
    def dropout_layer(self, bottom, keep_prob):
        if self.is_train == True:
            return tf.nn.dropout(bottom, keep_prob=keep_prob)
        else:
            return bottom

    def idp_conv_bn_layer(self, bottom, name, dp=1.0):
        with tf.name_scope(name+str(int(dp*100))):
            with tf.variable_scope(self.scope_name,reuse=True):
                conv_filter = tf.get_variable(name=name+"_W")
                conv_biases = tf.get_variable(name=name+"_b")
                conv_gamma  = tf.get_variable(name=name+"_gamma")
                moving_mean = tf.get_variable(name=name+'_bn_mean')
                moving_variance = tf.get_variable(name=name+'_bn_variance')
                beta = tf.get_variable(name=name+'_beta')
            H,W,C,O = conv_filter.get_shape().as_list()
            print(bottom.get_shape().as_list()) 
            if name is not 'conv1_1':
                bottom = bottom[:,:,:,:int(C*dp)]
                print("AFTER",bottom.get_shape().as_list())
                conv_filter = conv_filter[:,:,:int(C*dp),:]

            # create a mask determined by the dot product percentage
            n1 = int(O * dp)
            n0 = O - n1
            mask = tf.constant(value=np.append(np.ones(n1, dtype='float32'), np.zeros(n0, dtype='float32')), dtype=tf.float32)
            conv_gamma = tf.multiply(conv_gamma, mask)
            beta = tf.multiply(beta, mask)
            
            conv = tf.nn.conv2d(bottom, conv_filter, [1, 1, 1, 1], padding='SAME')
            conv = tf.nn.bias_add(conv, conv_biases)

            from tensorflow.python.training.moving_averages import assign_moving_average
            def mean_var_with_update():
                mean, variance = tf.nn.moments(conv, [0,1,2], name='moments')
                with tf.control_dependencies([assign_moving_average(moving_mean, mean, 0.99),
                                              assign_moving_average(moving_variance, variance, 0.99)]):
                    return tf.identity(mean), tf.identity(variance)

            mean, variance = tf.cond(self.is_train, mean_var_with_update, lambda:(moving_mean, moving_variance))

            conv = tf.nn.batch_normalization(conv, mean, variance, beta, conv_gamma, 1e-05)
            relu = tf.nn.relu(conv)
            
            return relu

    def fc_layer(self, bottom, name):
        with tf.name_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])
            
            with tf.variable_scope(self.scope_name,reuse=True):
                weights = tf.get_variable(name=name+"_W")
                biases = tf.get_variable(name=name+"_b")

            # Fully connected layer. Note that the '+' operation automatically broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)
            return fc

    def get_conv_filter(self, name, new_bn=False, shape=None):
        if shape is not None:
            conv_filter = tf.get_variable(shape=shape, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name=name+"_W", dtype=tf.float32)
        elif name in self.data_dict.keys():
            conv_filter = tf.get_variable(initializer=self.data_dict[name][0], name=name+"_W")
        else:
            print("please specify a name in data_dict or specify a shape in use")

        H,W,C,O = conv_filter.get_shape().as_list()

        if name+"_gamma" in self.data_dict.keys() and not new_bn: 
            gamma = tf.get_variable(initializer=self.data_dict[name+"_gamma"], name=name+"_gamma")
        else:
            gamma = tf.get_variable(initializer=self.get_profile(O, self.prof_type), name=name+"_gamma")

        if name+"_beta" in self.data_dict.keys() and not new_bn:
            beta = tf.get_variable(initializer=self.data_dict[name+"_beta"], name=name+"_beta")
        else:
            beta = tf.get_variable(shape=(O,), initializer=tf.zeros_initializer(), name=name+'_beta')

        if name+"_bn_mean" in self.data_dict.keys() and not new_bn:
            bn_mean = tf.get_variable(initializer=self.data_dict[name+"_bn_mean"], name=name+"_bn_mean")
        else:
            bn_mean = tf.get_variable(shape=(O,), initializer=tf.zeros_initializer(), name=name+'_bn_mean')

        if name+"_bn_variance" in self.data_dict.keys() and not new_bn: 
            bn_variance = tf.get_variable(initializer=self.data_dict[name+"_bn_variance"], name=name+"_bn_variance")
        else:
            bn_variance = tf.get_variable(shape=(O,),initializer=tf.ones_initializer(), name=name+'_bn_variance')
        
        return conv_filter, gamma, beta, bn_mean, bn_variance

    def get_fc_layer(self, name, shape=None):
        if shape is not None:
            return tf.get_variable(shape=shape, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name=name+"_W", dtype=tf.float32)
        elif name in self.data_dict.keys():
            return tf.get_variable(initializer=self.data_dict[name][0], name=name+"_W", dtype=tf.float32)
        else:
            print("please specify a name in data_dict or specify a shape in use")
            return None
            
    def get_bias(self, name, shape=None):
        if shape is not None:
            return tf.get_variable(shape=shape, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name=name+"_b", dtype=tf.float32)
        elif name in self.data_dict.keys():
            return tf.get_variable(initializer=self.data_dict[name][1], name=name+"_b", dtype=tf.float32)
        else:
            print("please specify a name in data_dict or specify a shape in use")
            return None

    def get_profile(self, C, prof_type):
        def half_exp(n, k=1, dtype='float32'):
            n_ones = int(n/2)
            n_other = n - n_ones
            return np.append(np.ones(n_ones, dtype=dtype), np.exp((1-k)*np.arange(n_other), dtype=dtype))
        if prof_type == "linear":
            profile = np.linspace(2.0,0.0, num=C, endpoint=False, dtype='float32')
        elif prof_type == "all-one":
            profile = np.ones(C, dtype='float32')
        elif prof_type == "half-exp":
            profile = half_exp(C, 2.0)
        elif prof_type == "harmonic":
            profile = np.array(1.0/(np.arange(C)+1), dtype='float32')
        else:
            raise ValueError("prof_type must be \"all-one\", \"half-exp\", \"harmonic\" or \"linear\".")
        return profile
                


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import pandas as pd
import skimage.io as imageio
import pickle

In [3]:
from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display

In [4]:
with open('save/label_dict.pkl', 'rb') as f:
    y_dict = pickle.load(f)

In [7]:
HOME_DIR = "/data/put_data/cmchang/DLCV_final/"
TRAIN_DIR = HOME_DIR+"dlcv_final_2_dataset/train/"
VALID_DIR = HOME_DIR+"dlcv_final_2_dataset/val/"

In [8]:
dtrain = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/train_id.txt", header=None,sep=" ", names=["img", "id"])
dvalid = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/val_id.txt", header=None,sep=" ", names=["img", "id"])

In [9]:
train_list = list(TRAIN_DIR+dtrain.img)
valid_list = list(VALID_DIR+dvalid.img)

In [10]:
def readImgList(file_list):
    images = list()
    for i, file in enumerate(file_list):
        print(i, end="\r")
        img = imageio.imread(file)
        img = img.astype(int)
        images.append(img)
    return np.array(images)

In [11]:
def transformLabel(id_list, y_dict):
    label = list()
    for uid in list(id_list):
        label.append(y_dict[uid])
    return np.array(label)

In [12]:
def one_hot_encoding(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

In [13]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [14]:
Xtrain = readImgList(train_list)

In [15]:
Xvalid = readImgList(valid_list)

In [16]:
Xtrain.shape

(56475, 218, 178, 3)

In [17]:
ytrain = transformLabel(list(dtrain.id), y_dict)

In [18]:
yvalid = transformLabel(list(dvalid.id), y_dict)

In [19]:
Ytrain = one_hot_encoding(ytrain, len(y_dict))
Yvalid = one_hot_encoding(yvalid, len(y_dict))

In [20]:
scope_name = "Model"

In [21]:
model = VGG16(scope_name=scope_name)

In [22]:
FLAG_init_from = "save_all-one_2/para_dict.npy"
FLAG_prof_type = "linear"
FLAG_lambda_s = 2e-3
FLAG_lambda_m = 1e-3
FLAG_decay = 1e-5
FLAG_lr = 2e-4
FLAG_keep_prob = 1.0
FLAG_save_dir = "finetune_v3_from_save_all-one_2/"

In [23]:
model.build(vgg16_npy_path=FLAG_init_from,
            shape=Xtrain.shape[1:],
            classes=len(y_dict),
            prof_type=FLAG_prof_type,
            conv_pre_training=True,
            fc_pre_training=True,
            new_bn=True)

build model started
npy file loaded
build model finished: 2s


In [24]:
model.sparsity_train(l1_gamma=FLAG_lambda_s, l1_gamma_diff=FLAG_lambda_m, decay=FLAG_decay, keep_prob=FLAG_keep_prob)

[None, 218, 178, 3]
[None, 218, 178, 64]
AFTER [None, 218, 178, 64]
[None, 109, 89, 64]
AFTER [None, 109, 89, 64]
[None, 109, 89, 128]
AFTER [None, 109, 89, 128]
[None, 55, 45, 128]
AFTER [None, 55, 45, 128]
[None, 55, 45, 256]
AFTER [None, 55, 45, 256]
[None, 55, 45, 256]
AFTER [None, 55, 45, 256]
[None, 28, 23, 256]
AFTER [None, 28, 23, 256]
[None, 28, 23, 512]
AFTER [None, 28, 23, 512]
[None, 28, 23, 512]
AFTER [None, 28, 23, 512]
[None, 14, 12, 512]
AFTER [None, 14, 12, 512]
[None, 14, 12, 512]
AFTER [None, 14, 12, 512]
[None, 14, 12, 512]
AFTER [None, 14, 12, 512]
sparsity train operation setup: 1s


In [25]:
import imgaug as ia
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
transform = iaa.Sequential([
    sometimes(iaa.Affine(translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})),
    sometimes(iaa.Affine(scale={"x": (0.85, 1.15), "y":(0.85, 1.15)})),
    sometimes(iaa.Affine(rotate=(-45, 45))),
    sometimes(iaa.Fliplr(0.5))
])

In [26]:
print("===== create directory =====")
if not os.path.exists(FLAG_save_dir):
    os.makedirs(FLAG_save_dir)

arr_spareness = []

# define tasks
tasks = ['var_dp']
print(tasks)

# initial task
cur_task = tasks[0]
obj = model.loss_dict[tasks[0]]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

   # hyper parameters
    batch_size = 64
    epoch = 100
    early_stop_patience = 10
    min_delta = 0.0001
    opt_type = 'adam'

    # recorder
    epoch_counter = 0
    history = list()

    # optimizer
    global_step = tf.Variable(0, trainable=False)

    # Passing global_step to minimize() will increment it at each step.
    if opt_type is 'sgd':
        learning_rate = FLAG_lr # adam # 4e-3 #sgd
        #half_cycle = 20000
        #learning_rate = tf.train.exponential_decay(learning_rate, global_step, half_cycle, 0.5, staircase=True)
        opt = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)
    else:
        learning_rate = FLAG_lr # adam # 4e-3 #sgd
        #half_cycle = 10000
        #learning_rate = tf.train.exponential_decay(learning_rate, global_step, half_cycle, 0.5, staircase=True)
        opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)

    checkpoint_path = os.path.join(FLAG_save_dir, 'model.ckpt')
    tvars_trainable = tf.trainable_variables()

    train_vars = list()
    for var in tf.trainable_variables():
        if model.scope_name in var.name:
            train_vars.append(var)
            
    train_op = opt.minimize(obj, global_step=global_step, var_list=tvars_trainable)
    
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=len(tasks))

    # progress bar
    ptrain = IntProgress()
    pval = IntProgress()
    display(ptrain)
    display(pval)
    ptrain.max = int(Xtrain.shape[0]/batch_size)
    pval.max = int(Xvalid.shape[0]/batch_size)

    spareness = model.spareness(thresh=0.05)
    print("initial spareness: %s" % sess.run(spareness))

    # re-initialize
    initialize_uninitialized(sess)

    # reset due to adding a new task
    patience_counter = 0
    current_best_val_accu = 0

    # optimize when the aggregated obj
    while(patience_counter < early_stop_patience and epoch_counter < epoch):

        def load_batches():
            for i in range(int(Xtrain.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                batch = ia.Batch(images=Xtrain[st:ed,:,:,:], data=Ytrain[st:ed,:])
                yield batch

        batch_loader = ia.BatchLoader(load_batches)
        bg_augmenter = ia.BackgroundAugmenter(batch_loader=batch_loader, augseq=transform, nb_workers=8)

        # start training
        stime = time.time()
        bar_train = Bar('Training', max=int(Xtrain.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
        bar_val =  Bar('Validation', max=int(Xvalid.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
        train_loss, train_accu = 0.0, 0.0
        while True:
            batch = bg_augmenter.get_batch()
            if batch is None:
                print("Finished epoch.")
                break
            x_images_aug = batch.images_aug
            y_images = batch.data
            loss, accu, _ = sess.run([obj, model.accu_dict[cur_task], train_op], feed_dict={model.x: x_images_aug,
                            model.y: y_images,
                            model.is_train: True})
            bar_train.next()
            train_loss += loss
            train_accu += accu
            ptrain.value +=1
            ptrain.description = "Training %s/%s" % (ptrain.value, ptrain.max)
        train_loss = train_loss/ptrain.value
        train_accu = train_accu/ptrain.value
        batch_loader.terminate()
        bg_augmenter.terminate()

        # validation
        val_loss = 0
        val_accu = 0
        for i in range(int(Xvalid.shape[0]/batch_size)):
            st = i*batch_size
            ed = (i+1)*batch_size
            loss, accu = sess.run([obj, model.accu_dict[cur_task]],
                                feed_dict={model.x: Xvalid[st:ed,:],
                                            model.y: Yvalid[st:ed,:],
                                            model.is_train: False})
            val_loss += loss
            val_accu += accu
            pval.value += 1
            pval.description = "Testing %s/%s" % (pval.value, pval.value)
        val_loss = val_loss/pval.value
        val_accu = val_accu/pval.value

        print("\nspareness: %s" % sess.run(spareness))
        # early stopping check
        if (val_accu - current_best_val_accu) > min_delta:
            current_best_val_accu = val_accu
            patience_counter = 0

            para_dict = sess.run(model.para_dict)
            np.save(os.path.join(FLAG_save_dir, "para_dict.npy"), para_dict)
            print("save in %s" % os.path.join(FLAG_save_dir, "para_dict.npy"))
        else:
            patience_counter += 1

        # shuffle Xtrain and Ytrain in the next epoch
        idx = np.random.permutation(Xtrain.shape[0])
        Xtrain, Ytrain = Xtrain[idx,:,:,:], Ytrain[idx,:]

        # epoch end
        # writer.add_summary(epoch_summary, epoch_counter)
        epoch_counter += 1

        ptrain.value = 0
        pval.value = 0
        bar_train.finish()
        bar_val.finish()

        print("Epoch %s (%s), %s sec >> train loss: %.4f, train accu: %.4f, val loss: %.4f, val accu at %s: %.4f" % (epoch_counter, patience_counter, round(time.time()-stime,2), train_loss, train_accu, val_loss, cur_task, val_accu))
        history.append([train_loss, train_accu, val_loss, val_accu ])
        
        if epoch_counter % 10 == 0:
            import matplotlib.pyplot as plt
            df = pd.DataFrame(history)
            df.columns = ['train_loss', 'train_accu', 'val_loss', 'val_accu']
            df[['train_loss', 'val_loss']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'loss.png'))
            df[['train_accu', 'val_accu']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'accu.png'))
        

===== create directory =====
['var_dp']


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


initial spareness: 0.023200758
Finished epoch.

spareness: 0.02935606
save in finetune_v3_from_save_all-one_2/para_dict.npy
Epoch 1 (0), 1757.09 sec >> train loss: 11.3799, train accu: 0.6780, val loss: 15.7223, val accu at var_dp: 0.2031
Finished epoch.

spareness: 0.03787879
save in finetune_v3_from_save_all-one_2/para_dict.npy
Epoch 2 (0), 1482.95 sec >> train loss: 10.7967, train accu: 0.7067, val loss: 12.6855, val accu at var_dp: 0.4844
Finished epoch.


Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 247, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 398, in _send_bytes
    self._send(buf)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe



spareness: 0.049952652
Epoch 3 (1), 2525.36 sec >> train loss: 10.2411, train accu: 0.7557, val loss: 12.1805, val accu at var_dp: 0.4674
Finished epoch.

spareness: 0.062973484
Epoch 4 (2), 3072.88 sec >> train loss: 9.7490, train accu: 0.7835, val loss: 12.2740, val accu at var_dp: 0.4413
Finished epoch.

spareness: 0.07694129
save in finetune_v3_from_save_all-one_2/para_dict.npy
Epoch 5 (0), 1504.04 sec >> train loss: 9.2607, train accu: 0.8095, val loss: 11.3590, val accu at var_dp: 0.5038
Finished epoch.

spareness: 0.09019887
Epoch 6 (1), 2233.04 sec >> train loss: 8.8277, train accu: 0.8223, val loss: 11.1932, val accu at var_dp: 0.4803
Finished epoch.

spareness: 0.10440341
save in finetune_v3_from_save_all-one_2/para_dict.npy
Epoch 7 (0), 2102.07 sec >> train loss: 8.3726, train accu: 0.8439, val loss: 10.6124, val accu at var_dp: 0.5067
Finished epoch.

spareness: 0.11813447
save in finetune_v3_from_save_all-one_2/para_dict.npy
Epoch 8 (0), 2362.74 sec >> train loss: 7.9752,

Finished epoch.

spareness: 0.5814394
Epoch 51 (4), 1649.86 sec >> train loss: 1.0426, train accu: 0.9789, val loss: 2.3643, val accu at var_dp: 0.7309
Finished epoch.

spareness: 0.59138256
Epoch 52 (5), 2164.83 sec >> train loss: 1.0046, train accu: 0.9805, val loss: 2.2546, val accu at var_dp: 0.7444
Finished epoch.

spareness: 0.5999053
save in finetune_v3_from_save_all-one_2/para_dict.npy
Epoch 53 (0), 2101.18 sec >> train loss: 0.9814, train accu: 0.9787, val loss: 2.1854, val accu at var_dp: 0.7525
Finished epoch.

spareness: 0.6098485
Epoch 54 (1), 1868.99 sec >> train loss: 0.9506, train accu: 0.9805, val loss: 2.2443, val accu at var_dp: 0.7363
Finished epoch.

spareness: 0.6216856
Epoch 55 (2), 2241.12 sec >> train loss: 0.9266, train accu: 0.9796, val loss: 2.1385, val accu at var_dp: 0.7482
Finished epoch.

spareness: 0.63352275
Epoch 56 (3), 2087.54 sec >> train loss: 0.9024, train accu: 0.9800, val loss: 2.1358, val accu at var_dp: 0.7489
Finished epoch.

spareness: 0.64

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 247, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 398, in _send_bytes
    self._send(buf)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe



spareness: 0.86174244
Epoch 74 (2), 1985.76 sec >> train loss: 0.6276, train accu: 0.9845, val loss: 1.9210, val accu at var_dp: 0.7374
Finished epoch.

spareness: 0.8655303
Epoch 75 (3), 2161.05 sec >> train loss: 0.6271, train accu: 0.9819, val loss: 1.7908, val accu at var_dp: 0.7649
Finished epoch.

spareness: 0.8733428
Epoch 76 (4), 2223.72 sec >> train loss: 0.6077, train accu: 0.9854, val loss: 1.7762, val accu at var_dp: 0.7645
Finished epoch.

spareness: 0.87736744
Epoch 77 (5), 2077.18 sec >> train loss: 0.6046, train accu: 0.9842, val loss: 1.8439, val accu at var_dp: 0.7503
Finished epoch.

spareness: 0.883286
Epoch 78 (6), 2052.37 sec >> train loss: 0.5968, train accu: 0.9852, val loss: 1.8432, val accu at var_dp: 0.7644
Finished epoch.

spareness: 0.88825756
Epoch 79 (7), 2255.95 sec >> train loss: 0.5899, train accu: 0.9841, val loss: 2.0743, val accu at var_dp: 0.7125
Finished epoch.

spareness: 0.89346594
Epoch 80 (8), 2243.58 sec >> train loss: 0.5861, train accu: 0.

NameError: name 'gammaSparsifyVGG16' is not defined